Ranking the rules

In [3]:
import pandas as pd
df = pd.read_csv('final_dataset.csv')
df.head()


,_id,user_id,review_count,review_id,text,date,stars,useful,funny,sentiment_score,...,rule5_threshold,rule2_threshold,rule4_threshold,rule3_threshold,similarity_score,rule6_threshold,count_ones,target,funny_threshold,star_threshold
0,661a5d4bee4d349db0518760,qVc8ODYU5SZjKXVBgXdI7w,585,Egy2a4qZeXGr2aY6KMxxbg,Remarkable food with beach access for the whol...,2017-05-09 23:21:36,5,0,0,0.405455,...,0,0,1,1,1.0,1,4,genuine,0,1
1,661a5d4bee4d349db0518761,qVc8ODYU5SZjKXVBgXdI7w,585,01vN0q6aMlFio6HAjLZz7Q,I loved everything about this lovely train sta...,2009-05-01 02:00:03,5,30,7,0.598016,...,1,0,1,1,1.0,1,5,genuine,1,1
2,661a5d4bee4d349db0518762,j14WgRoU_-2ZE1aw1dXrJg,4333,iMpqDa0Oyukiw5406KYNRw,The Praline Connection makes a mean po' boy. ...,2010-10-11 19:24:56,3,3,1,0.024826,...,1,1,1,1,1.0,1,5,fake,1,0
3,661a5d4bee4d349db0518763,j14WgRoU_-2ZE1aw1dXrJg,4333,3pCHQ8YHkuaZFFAEz7pz6A,"We walked over to Tennessee Brew Works, one of...",2018-10-23 14:24:32,4,12,5,0.144998,...,1,1,1,1,1.0,1,6,genuine,1,1
4,661a5d4bee4d349db0518764,j14WgRoU_-2ZE1aw1dXrJg,4333,FvTfqugdlzQSvFB1RAafNQ,"Logan Circle, also known as Logan Square, is a...",2019-08-04 12:29:33,4,5,0,0.140657,...,1,1,1,1,1.0,1,6,genuine,0,1


In [4]:
df.columns

Index(['_id', 'user_id', 'review_count', 'review_id', 'text', 'date', 'stars',
       'useful', 'funny', 'sentiment_score', 'rule1_threshold',
       'rule5_threshold', 'rule2_threshold', 'rule4_threshold',
       'rule3_threshold', 'similarity_score', 'rule6_threshold', 'count_ones',
       'target', 'funny_threshold', 'star_threshold'],
      dtype='object')

In [7]:
import pandas as pd

# Load the dataset
df = pd.read_csv('final_dataset.csv')

# Drop the remaining columns
columns_to_keep = ['user_id', 'review_id', 'text', 'date', 'stars', 'useful', 'funny', 'target']
df.drop(columns=df.columns.difference(columns_to_keep), inplace=True)

# Show the DataFrame with the dropped columns
print(df)


                      user_id               review_id  \
0      qVc8ODYU5SZjKXVBgXdI7w  Egy2a4qZeXGr2aY6KMxxbg   
1      qVc8ODYU5SZjKXVBgXdI7w  01vN0q6aMlFio6HAjLZz7Q   
2      j14WgRoU_-2ZE1aw1dXrJg  iMpqDa0Oyukiw5406KYNRw   
3      j14WgRoU_-2ZE1aw1dXrJg  3pCHQ8YHkuaZFFAEz7pz6A   
4      j14WgRoU_-2ZE1aw1dXrJg  FvTfqugdlzQSvFB1RAafNQ   
...                       ...                     ...   
53455  UKg3ybTCwNQJHFy-9vJJPQ  FpJMXcuWMXKg0Au9lEhfxw   
53456  1LS-1LbfUnLrftzcuDnalA  DJ79msGWItWK2ntfmVawuQ   
53457  yL4whR1Qb63FptG9IVRdVQ  5X1rxMbWQVGOQwNe0MWl8A   
53458  IWze1TqgVsA--cRLbU_nDQ  DYfFtk2BEuQk9qTv5eEfHw   
53459  PZBexV9rbHVNN31PwZ7w5g  uKvamuu3XMl32My29s_R4g   

                                                    text                 date  \
0      Remarkable food with beach access for the whol...  2017-05-09 23:21:36   
1      I loved everything about this lovely train sta...  2009-05-01 02:00:03   
2      The Praline Connection makes a mean po' boy.  ...  2010-10-11 19:

In [6]:
df.columns

Index(['user_id', 'review_id', 'text', 'date', 'stars', 'useful', 'funny',
       'target'],
      dtype='object')